# Вычислительная математика. Лабораторная работа 2: решение СЛАУ

In [10]:
from matplotlib import pyplot as plt
import numpy as np
import math as m
%matplotlib inline

In [463]:
A = np.matrix("6, 1; 1, 8")
B = (np.matrix("8, 17")).transpose()
print(f"A = {A}\nB = {B}")

A = [[6 1]
 [1 8]]
B = [[ 8]
 [17]]


In [457]:
def condition_number(A):
    print("condition number via norm-1", np.linalg.cond(A, 1))
    print("condition number via norm-2", np.linalg.cond(A))
    
condition_number(A)

condition number via norm-1 1.7234042553191486
condition number via norm-2 1.5063614866643267


In [464]:
# метод Якоби

def jacobi(A, b):
    
    epsilon = 0.0001
    count = len(b)
    
    E = np.identity(count)
    D = np.diagflat(A.diagonal())
    D_inverse = np.linalg.inv(D)
    B = E - D_inverse @ A
#     check_jacobi(B)
#     dd(A)
    g = D_inverse @ b
    
    MAX_ITER = 150
    Xk = np.matrix(np.zeros(count)).transpose()
    
    for i in range (MAX_ITER):
        Xk1 = B @ Xk + g
        if (max(abs(Xk - Xk1))).item() < epsilon:
            print(f"Needed {i + 1} iterations")
            break
        Xk = Xk1
        if i == MAX_ITER - 1:
            print(f"Maximum iterations reached: {i + 1}")
    
    return Xk
        
    
def check_jacobi(B):
    norm1 = np.linalg.norm(B, 1)
    norm3 = np.linalg.norm(B, np.inf)
    norm2 = np.linalg.norm(B, 2)
    if norm1 < 1:
        print(f"1 норма B = {norm1}; метод сходится")
    elif norm2 < 1:
        print(f"3 норма B = {norm2}; метод сходится")
    elif (norm3 < 1):
        print(f"2 норма B = {norm3}; метод сходится")
    else:
        print(f"норма B = {min(norm1, norm2, norm3)}; метод не сходится")
        
        
def dd(X):
    D = np.diag(np.abs(X)) # Find diagonal coefficients
    S = np.sum(np.abs(X), axis=1) - D # Find row sum without diagonal
    if np.all(D >= S):
        print ('есть диагональное преобладание')
    else:
        print ('нет диагонального преобладания')
    return
    
print(jacobi(A, B))
# nice_print_solutions(jacobi(A, B))

Needed 7 iterations
[[0.99999096]
 [1.99998192]]


In [459]:
# метод Зейделя
def zeidel(A, b):
    epsilon = 0.0001

    count = len(b)
    L = np.tril(A, k=-1)
    U = np.triu(A, k=1)
    D = np.diagflat(A.diagonal())
    
#     check_zeidel(L, D, U)
    
    diagonal_elems = np.diag(A)
    d = np.true_divide(b.transpose(), diagonal_elems).transpose() 
    
    C = - (A.transpose() / diagonal_elems).transpose() 
    np.fill_diagonal(C, 0)
        
    MAX_ITER = 150
    Xk = np.matrix(np.zeros(count)).transpose()
    
    for i in range(MAX_ITER):
        Xk1 = np.matrix(np.zeros(count)).transpose()
        for i in range(count):
            Xk1[i] = d[i]
            for j in range(0, i):
                Xk1[i] += C[i, j] * Xk1[j]
            for j in range(i, count):
                Xk1[i] += C[i, j] * Xk[j]
        if (max(abs(Xk - Xk1))).item() < epsilon:
            print(f"Needed {i + 1} iterations")
            break
        Xk = Xk1
        if i == MAX_ITER - 1:
            print("Maximum iterations reached")

    return Xk
    
def check_zeidel(L, D, U):
    B = -np.linalg.inv(L + D) @ U
    
    norm1 = np.linalg.norm(B, 1)
    norm3 = np.linalg.norm(B, np.inf)
    norm2 = np.linalg.norm(B, 2)
    if norm1 < 1:
        print(f"норма -(L + D)^-1 * U = {norm1}; метод сходится")
    elif norm2 < 1:
        print(f"норма -(L + D)^-1 * U = {norm2}; метод сходится")
    elif (norm3 < 1):
        print(f"норма -(L + D)^-1 * U = {norm3}; метод сходится")
    else:
        print(f"норма -(L + D)^-1 * U = {min(norm1, norm2, norm3)}; метод не сходится")
    
print(zeidel(A, B))
# nice_print_solutions(zeidel(A, B))

Needed 2 iterations
[[1.00000301]
 [1.99999962]]


In [460]:
# метод трехдиагональной прогонки

def get_left_v(A, B):
    return ((A[0, 1]) / (-A[0, 0])).item()

def get_left_u(A, B):
    return (-B[0, 0] / (-A[0, 0])).item()

def get_right_u(A, B, u, v, i):
    return ((A[i, i-1] * u[i-1] - B[i, 0])/(-A[i, i] - A[i, i-1] * v[i-1])).item()

def get_middle_v(A, B, u, v, i):
    return ((A[i, i+1])/(-A[i, i] - A[i, i-1] * v[i-1])).item()

def get_middle_u(A, B, u, v, i):
    return ((A[i, i-1] * u[i-1] - B[i])/(-A[i, i] - A[i, i-1] * v[i-1])).item()



def tridiagonal(A, B):
    rows = A.shape[0]
    columns = A.shape[1]
    
    dd(A)
    
    v = []
    u = []
    x = []
    
    # вперед
    for i in range(rows):
        if i == 0:
            v.append(get_left_v(A, B))
            u.append(get_left_u(A, B))
        elif i == rows - 1:
            v.append(0)
            u.append(get_right_u(A, B, u, v, i))
        else:
            v.append(get_middle_v(A, B, u, v, i))
            u.append(get_middle_u(A, B, u, v, i))
    
    # назад и вычисляем результат
    x.append(u[rows - 1])

    for j in range(rows - 1, 0, -1):
        s = x[rows - 1 - j] * v[j - 1] + u[j - 1]
        x.append(s)
    
    x.reverse()
    return x

print(tridiagonal(A, B))
# nice_print_solutions(tridiagonal(A, B))

есть диагональное преобладание
[1.0, 2.0]


In [350]:
def dot(A, B):
    res = 0
    for i in range(len(A)):
        res += A[i] * B[i]
    return res

In [451]:
A = np.matrix("1, 0.99; 0.99, 1")
B = (np.matrix("1, 1")).transpose()
print(f"A = {A}\nB = {B}")

A = [[1.   0.99]
 [0.99 1.  ]]
B = [[1]
 [1]]


In [330]:
A = np.matrix("100, 99; 99, 98")
B = (np.matrix("199, 197")).transpose()
print(f"A = {A}\nB = {B}")

A = [[100  99]
 [ 99  98]]
B = [[199]
 [197]]


In [465]:
# Метод наискорейшего спуска

def descend(A, b):
    
    print(check_symmetric(A), is_real(A), is_pos_def(A))
    
    epsilon = 0.0001
    count = len(b)

    MAX_ITER = 10
    Xk = np.matrix(np.zeros(count)).transpose()
    for i in range(MAX_ITER):
        rn = (A @ Xk - b)
#         print(rn, dot(rn, rn), (dot((A @ rn), rn)))
        alpha_double = dot(rn, rn) / (dot((A @ rn), rn))
        Xk1 = Xk - (rn * alpha_double)
#         print(Xk1, Xk)
        
        if (max(abs(Xk - Xk1))).item() < epsilon:
            print(f"Needed {i + 1} iterations")
            break
        Xk = Xk1
        if i == MAX_ITER - 1:
            print("Maximum iterations reached")
            
    return Xk

print(descend(A, B))  
# nice_print_solutions(descend(A, B))

матрица симметрична
матрица вещественна
матрица положительно определена
True True True
Needed 4 iterations
[[0.99998171]
 [2.00000826]]


In [466]:
# Метод минимальных невязок

def residuals(A, b):
    
#     print(check_symmetric(A), is_real(A), is_pos_def(A))
    
    epsilon = 0.0001
    count = len(b)

    MAX_ITER = 10
    Xk = np.matrix(np.zeros(count)).transpose()
    for i in range(MAX_ITER):
        rn = (A @ Xk - b)
        alpha_double = dot((A @ rn), rn) / dot((A @ rn), (A @ rn))
        Xk1 = Xk - (rn * alpha_double)
        
        if (max(abs(Xk - Xk1))).item() < epsilon:
            print(f"Needed {i + 1} iterations")
            break
        Xk = Xk1
        if i == MAX_ITER - 1:
            print("Maximum iterations reached")
            
    return Xk

print(residuals(A, B))   
# nice_print_solutions(residuals(A, B))

Needed 4 iterations
[[0.99998778]
 [2.00000536]]


In [271]:
def nice_print_solutions(X):
    X = np.array(X)
    print("Solutions:")
    for i in range (len(X)):
        print("x{} = {%.6f}".format(i, X[i]))

In [399]:
def is_pos_def(x):
    if np.all(np.linalg.eigvals(x) > 0):
        print("матрица положительно определена")
    else:
        print("матрица не положительно определена")
    return np.all(np.linalg.eigvals(x) > 0)

def check_symmetric(a, tol=1e-8):
    if np.all(np.abs(a-a.T) < tol):
        print("матрица симметрична")
    else: print("матрица несимметрична")
        
    return np.all(np.abs(a-a.T) < tol)

def is_real(x):
    if np.all(np.isreal(x)):
        print("матрица вещественна")
        return True
    else:
        print("матрица не вещественна")
    return False

In [402]:
print(check_symmetric(A), is_real(A), is_pos_def(A))

матрица симметрична
матрица вещественна
матрица положительно определена
True True True
